In [3]:
import autokeras as ak
from openml import datasets
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split, KFold
import pandas as pd
import numpy as np
import json
import time
from tensorflow.keras.models import load_model
import shutil
import os

2022-06-08 21:38:19.756454: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-08 21:38:19.756543: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [30]:
def run2(X : pd.DataFrame,y : pd.DataFrame):
    
    kfold = KFold(n_splits=10, shuffle=True)
    
    input_node = ak.StructuredDataInput()
    output_node = ak.StructuredDataBlock(categorical_encoding=True, normalize = True)(input_node)
    output_node = ak.DenseBlock()(output_node)
    output_node = ak.ClassificationHead(num_classes = 2, metrics = 'AUC')(output_node)
    clf = ak.AutoModel(
        inputs=input_node, outputs=output_node, overwrite=True, max_trials=10
    )
    
  
    best_auc = 0
    
    for fold, (train_index, test_index) in enumerate(kfold.split(X,y)):  # crossvalidation
        # if _del and os.path.isdir('structured_data_classifier'):
        #     shutil.rmtree(r'structured_data_classifier')
            
        
        train_X, test_X = X.iloc[train_index], X.iloc[test_index]
        train_y, test_y = y.iloc[train_index], y.iloc[test_index]
        
        clf.fit(train_X, train_y, epochs = 100)
        prediction = pd.DataFrame(clf.predict(test_X))
        test_y = test_y.apply(pd.to_numeric, errors='coerce').fillna(test_y) # problems with types...
        prediction = prediction.apply(pd.to_numeric, errors='coerce').fillna(prediction)  
        
        #scores
        
        auc = roc_auc_score(test_y,prediction)
        
        if auc > best_auc:
            best_auc = auc
            best_model = clf
    


    
    return best_model

In [31]:
df_train = pd.read_csv('df_1_train.csv', index_col = 0)
df_test = pd.read_csv('df_1_test.csv',index_col = 0)

In [32]:
df_train['Y'] = df_train['Y'].map({"Bad":0,"Good":1})  # bez mapowania nie ogarnia niektórych metryk

In [33]:
y = df_train['Y']
X = df_train.drop(columns = 'Y')

In [34]:
clf = run2(X,y)

Trial 10 Complete [00h 00m 31s]
val_loss: 0.5561140775680542

Best val_loss So Far: 0.5529577732086182
Total elapsed time: 00h 05m 41s
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
208/208 [==============================] - 3s 8ms/step - loss: 0.6852 - auc: 0.6595
Epoch 2/100
208/208 [==============================] - 2s 8ms/step - loss: 0.6384 - auc: 0.7005
Epoch 3/100
208/208 [==============================] - 2s 7ms/step - loss: 0.6368 - auc: 0.7016
Epoch 4/100
208/208 [==============================] - 2s 8ms/step - loss: 0.6358 - auc: 0.7063
Epoch 5/100
208/208 [==============================] - 1s 7ms/step - loss: 0.6331 - auc: 0.7119
Epoch 6/100
208/208 [==============================] - 2s 8ms/step - loss: 0.6291 - auc: 0.7172
Epoch 7/100
208/208 [==============================] - 2s 8ms/step - loss: 0.6325 - auc: 0.7153
Epoch 8/100
208/208 [==============================] - 2s 8ms/step - loss: 0.6308 - auc: 0.7154
Epoch 9/100
208/208 [==============================] - 1s 7

2022-06-08 21:51:16.113436: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets
24/24 [==============================] - 0s 3ms/step


In [35]:
model_export = clf.export_model()
prob = model_export.predict(df_test)

In [36]:
wynik = pd.DataFrame(columns = ["row_id", "prob"]) 
wynik["row_id"] = list(df_test.index)
wynik["prob"] = prob

In [37]:
wynik.to_csv("submission2.csv")